# The Kalman Filter 
is a mathematical algorithm that uses a series of measurements observed over time to estimate the true state of a system. It is widely used in various fields, including engineering, economics, and signal processing, for applications such as control systems, navigation, and time-series analysis. The Kalman Filter combines the measurements' uncertainty (noise) with the system's dynamics to produce a more accurate and optimal estimate of the true state, even in the presence of noise.

The Kalman Filter operates in two main steps: prediction and update.

Prediction:
In this step, the filter uses the previous estimate and the system's dynamics model to predict the current state's value. This prediction includes accounting for the inherent uncertainty in the prediction due to measurement noise and the dynamics of the system.

Update:
In this step, the filter incorporates the most recent measurement to update the predicted state estimate. The update process considers both the prediction's uncertainty and the measurement's uncertainty to produce a more refined and accurate state estimate.

The Kalman Filter is particularly effective when dealing with noisy measurements and systems with unknown or changing dynamics. It's also capable of handling missing or irregularly spaced measurements. The filter is recursive, meaning it can process new measurements in real-time without needing to reprocess the entire historical data.

In [9]:
pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=4d06a8552dc0025a6f13b76e41b8a33744052c73205a9a88dfc0c2c8d66788dc
  Stored in directory: /Users/chandrajason/Library/Caches/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy
Note: you may need to restart the kernel to use updated packages.


In [4]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter
import numpy as np

# Fetch historical price data from Yahoo Finance
ticker_symbol = 'AAPL'
start_date = '2022-01-01'
end_date = '2022-12-31'
data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Extract the 'Close' prices
true_values = data['Close'].values

# Generate noisy measurements using random noise
np.random.seed(0)
noisy_measurements = true_values + np.random.normal(0, 5, len(true_values))

# Create a Kalman filter
kf = KalmanFilter(dim_x=2, dim_z=1)
kf.x = np.array([0, 0])  # initial state [position, velocity]
kf.F = np.array([[1, 1], [0, 1]])  # state transition matrix
kf.H = np.array([[1, 0]])  # observation matrix
kf.P *= 1000  # initial covariance matrix
kf.R = 25  # measurement noise covariance
kf.Q = np.array([[0.001, 0], [0, 0.001]])  # process noise covariance

# Apply the filter to noisy measurements
(filtered_state_means, _) = kf.batch_filter(noisy_measurements)

# Create a DataFrame with the original data, noisy measurements, and filtered results
data = pd.DataFrame({'True Values': true_values, 'Noisy Measurements': noisy_measurements, 'Filtered Results': filtered_state_means[:, 0]})

# Calculate moving averages of different window sizes
data['MA_1m'] = data['Noisy Measurements'].rolling(window=1).mean()
data['MA_2m'] = data['Noisy Measurements'].rolling(window=2).mean()
data['MA_3m'] = data['Noisy Measurements'].rolling(window=3).mean()

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(data.index, data['True Values'], label='True Values', color='blue')
plt.plot(data.index, data['Noisy Measurements'], label='Noisy Measurements', color='red', alpha=0.5)
plt.plot(data.index, data['Filtered Results'], label='Filtered Results', color='green')
plt.plot(data.index, data['MA_1m'], label='1-Month Moving Average', color='orange', linestyle='--')
plt.plot(data.index, data['MA_2m'], label='2-Month Moving Average', color='purple', linestyle='--')
plt.plot(data.index, data['MA_3m'], label='3-Month Moving Average', color='brown', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title(f'{ticker_symbol} Stock Price with Kalman Filter and Moving Averages')
plt.legend()
plt.tight_layout()
plt.show()


[*********************100%%**********************]  1 of 1 completed


ValueError: object has no __init__ method